Going to use this code to sorta-kinda start to conduct correlation analyses between the elliptical metric/themselves/temperature.


In [1]:
import numpy as np
import math
import xarray as xr 
import pickle 
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
import scipy.stats

In [ ]:
infile = open("gph_97.p", 'rb')
gph = pickle.load(infile)
infile.close()

infile = open("gph_time_97.p", 'rb')
gph_time = pickle.load(infile)
infile.close()

infile = open("temp850_97.p", 'rb')
temp = pickle.load(infile)
infile.close()

infile = open("temp850_time_97.p", 'rb')
temp_time = pickle.load(infile)
infile.close()

infile = open("gph_time_97.p", 'rb')
gph_time = pickle.load(infile)
infile.close()

infile = open("gph_time_97.p", 'rb')
gph_time = pickle.load(infile)
infile.close()

infile = open("gph_lat_97.p", 'rb')
gph_lat = pickle.load(infile)
infile.close()

infile = open("gph_lon_97.p", 'rb')
gph_lon = pickle.load(infile)
infile.close()

In [ ]:
infile = open("../Clusters/Fall2023Clustering/UW_lat.p", 'rb')
unweighted_lat = pickle.load(infile)
infile.close()

infile = open("../Clusters/Fall2023Clustering/UW_lon.p", 'rb') 
unweighted_lon = pickle.load(infile)
infile.close()

In [ ]:
gph = xr.DataArray(data= gph, 
                       dims = ["time","lat","lon"],
                       coords = dict(
                           time = gph_time,
                           lat = unweighted_lat,
                           lon = unweighted_lon)
                      )
gph_D = gph.groupby('time.date').mean()

gph

<xarray.DataArray (time: 24320, lat: 128, lon: 512)>
array([[[30898.127, 30898.383, 30899.045, ..., 30899.605, 30898.791,
         30898.28 ],
        [30898.434, 30898.791, 30899.504, ..., 30899.76 , 30898.893,
         30898.434],
        [30898.484, 30899.146, 30899.912, ..., 30899.504, 30898.586,
         30898.28 ],
        ...,
        [29642.959, 29642.092, 29641.174, ..., 29645.764, 29644.846,
         29643.877],
        [29671.715, 29671.104, 29670.49 , ..., 29673.602, 29672.938,
         29672.326],
        [29701.693, 29701.389, 29701.133, ..., 29702.56 , 29702.256,
         29701.95 ]],

       [[30859.053, 30856.707, 30854.465, ..., 30865.223, 30863.438,
         30861.348],
        [30855.79 , 30853.445, 30851.303, ..., 30863.133, 30861.092,
         30858.441],
        [30853.037, 30850.793, 30848.906, ..., 30860.889, 30858.695,
         30855.842],
...
        [29473.12 , 29473.68 , 29474.24 , ..., 29471.588, 29472.098,
         29472.61 ],
        [29501.16 , 29501.467, 29501.824, ..., 29500.191, 29500.498,
         29500.803],
        [29530.63 , 29530.783, 29530.936, ..., 29530.223, 29530.324,
         29530.477]],

       [[30907.75 , 30908.31 , 30909.28 , ..., 30905.354, 30906.578,
         30907.24 ],
        [30905.1  , 30905.762, 30906.73 , ..., 30902.703, 30903.621,
         30904.334],
        [30902.295, 30903.06 , 30903.824, ..., 30900.459, 30900.918,
         30901.428],
        ...,
        [29456.133, 29456.643, 29457.154, ..., 29454.654, 29455.113,
         29455.623],
        [29484.482, 29484.787, 29485.094, ..., 29483.564, 29483.87 ,
         29484.176],
        [29514.46 , 29514.562, 29514.717, ..., 29514.053, 29514.156,
         29514.309]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1979-11-01 ... 2019-04-01T18:00:00
  * lat      (lat) float32 0.3509 1.053 1.754 2.456 ... 87.37 88.07 88.77 89.46
  * lon      (lon) float32 0.0 0.7031 1.406 2.109 ... 357.2 357.9 358.6 359.3

In [ ]:
#calculate daily mean values
daily_mean_gph = gph.groupby('time.dayofyear').mean()
#test gph anomaly
daily_anom_gph = gph.groupby('time.dayofyear') - daily_mean_gph

##straight gph cap values
g_cap = gph.sel(lat = slice(60,90))

#anomaly cap values
cap_mean_gph = g_cap.groupby('time.dayofyear').mean()
#test gph anomaly
cap_anom_gph = g_cap.groupby('time.dayofyear') - cap_mean_gph
cap_anom_gph

<xarray.DataArray (time: 24320, lat: 42, lon: 512)>
array([[[-174.88086 , -165.70508 , -156.59375 , ..., -202.39648 ,
         -193.33008 , -184.20508 ],
        [-184.66797 , -175.0625  , -165.5293  , ..., -212.70508 ,
         -203.48633 , -194.19531 ],
        [-191.7168  , -181.6582  , -171.60352 , ..., -220.43945 ,
         -210.9414  , -201.48633 ],
        ...,
        [ 316.20312 ,  315.67383 ,  315.08008 , ...,  317.99023 ,
          317.4121  ,  316.7832  ],
        [ 338.03906 ,  337.64844 ,  337.24805 , ...,  339.23828 ,
          338.79883 ,  338.4121  ],
        [ 358.73047 ,  358.51953 ,  358.35938 , ...,  359.2832  ,
          359.07812 ,  358.875   ]],

       [[-226.49805 , -213.3457  , -200.25586 , ..., -263.1914  ,
         -251.72852 , -239.49414 ],
        [-232.15625 , -219.13281 , -206.08203 , ..., -268.60547 ,
         -257.0918  , -244.89453 ],
        [-236.39844 , -223.79102 , -211.1875  , ..., -272.00586 ,
         -260.5703  , -248.66602 ],
...
        [-750.6836  , -749.7832  , -748.8906  , ..., -753.25586 ,
         -752.3926  , -751.52734 ],
        [-735.69336 , -735.16797 , -734.60156 , ..., -737.3281  ,
         -736.79297 , -736.25977 ],
        [-720.4121  , -720.1699  , -719.9336  , ..., -721.0996  ,
         -720.9219  , -720.66406 ]],

       [[ -76.9707  ,  -78.93164 ,  -80.65039 , ...,  -67.08789 ,
          -71.2168  ,  -74.46094 ],
        [-134.61914 , -135.69727 , -136.11914 , ..., -123.48242 ,
         -128.54102 , -132.29492 ],
        [-190.10352 , -191.54297 , -191.72852 , ..., -176.8711  ,
         -182.58984 , -187.11719 ],
        ...,
        [-767.6699  , -766.8203  , -765.97656 , ..., -770.18945 ,
         -769.37695 , -768.5137  ],
        [-752.3711  , -751.84766 , -751.33203 , ..., -753.9551  ,
         -753.4219  , -752.8867  ],
        [-736.58203 , -736.3906  , -736.15234 , ..., -737.26953 ,
         -737.08984 , -736.83203 ]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 1979-11-01 ... 2019-04-01T18:00:00
  * lat        (lat) float32 60.7 61.4 62.11 62.81 ... 87.37 88.07 88.77 89.46
  * lon        (lon) float32 0.0 0.7031 1.406 2.109 ... 357.2 357.9 358.6 359.3
    dayofyear  (time) int64 305 305 305 305 306 306 306 ... 90 90 90 91 91 91 91

In [ ]:
temp = xr.DataArray(data= temp, 
                       dims = ["time","lat","lon"],
                       coords = dict(
                           time = temp_time,
                           lat = unweighted_lat,
                           lon = unweighted_lon)
                      )
temp_D = gph.groupby('time.date').mean()

temp

<xarray.DataArray (time: 24320, lat: 128, lon: 512)>
array([[[289.2945 , 289.357  , 289.2652 , ..., 289.15973, 289.19293,
         289.14215],
        [289.59625, 289.68903, 289.58258, ..., 289.41754, 289.50153,
         289.45758],
        [289.64996, 289.6939 , 289.67535, ..., 289.72418, 289.7447 ,
         289.6734 ],
        ...,
        [253.96733, 253.97514, 253.98198, ..., 253.93999, 253.94975,
         253.95854],
        [253.44292, 253.4439 , 253.44292, ..., 253.43999, 253.44194,
         253.44292],
        [252.87163, 252.87163, 252.87163, ..., 252.87163, 252.87163,
         252.87163]],

       [[288.95602, 288.99802, 289.00778, ..., 288.91208, 288.93063,
         288.87888],
        [289.29977, 289.36325, 289.40915, ..., 289.22946, 289.26755,
         289.21872],
        [289.54   , 289.59372, 289.62497, ..., 289.48532, 289.39645,
         289.38864],
...
        [260.26007, 260.35773, 260.45538, ..., 259.96906, 260.0667 ,
         260.16437],
        [260.59015, 260.6507 , 260.71124, ..., 260.4046 , 260.4671 ,
         260.52765],
        [260.6292 , 260.65656, 260.68195, ..., 260.54913, 260.57648,
         260.60187]],

       [[293.04654, 293.08755, 293.06607, ..., 292.75357, 292.90396,
         293.04068],
        [293.0153 , 292.99966, 292.97232, ..., 292.8317 , 292.95474,
         293.06412],
        [293.1735 , 293.07193, 292.9977 , ..., 292.89615, 292.97037,
         293.13052],
        ...,
        [263.63248, 263.65005, 263.66763, ..., 263.56607, 263.5895 ,
         263.611  ],
        [263.44498, 263.44693, 263.44693, ..., 263.43716, 263.44107,
         263.44302],
        [262.99966, 262.9977 , 262.99576, ..., 263.00552, 263.00357,
         263.00162]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1979-11-01 ... 2019-04-01T18:00:00
  * lat      (lat) float32 0.3509 1.053 1.754 2.456 ... 87.37 88.07 88.77 89.46
  * lon      (lon) float32 0.0 0.7031 1.406 2.109 ... 357.2 357.9 358.6 359.3

In [ ]:
#calculate daily mean values
daily_mean_temp = temp.groupby('time.dayofyear').mean()
#test gph anomaly
daily_anom_temp = temp.groupby('time.dayofyear') - daily_mean_temp

##straight gph cap values
t_cap = temp.sel(lat = slice(60,90))
##great lakes
t_gl = temp.sel(lat = slice(40,50), lon = slice(270,290))

#anomaly cap values
cap_mean_t= t_cap.groupby('time.dayofyear').mean()
#test gph anomaly
cap_anom_t = t_cap.groupby('time.dayofyear') - cap_mean_t

#anomaly cap values
gl_mean_t= t_gl.groupby('time.dayofyear').mean()
#test gph anomaly
gl_anom_t = t_gl.groupby('time.dayofyear') - gl_mean_t
gl_anom_t

<xarray.DataArray (time: 24320, lat: 14, lon: 29)>
array([[[  2.1198425 ,   3.8211365 ,   5.0191956 , ...,   0.36218262,
           0.16464233,  -0.24267578],
        [  3.17453   ,   4.7858887 ,   5.843506  , ...,   0.52316284,
           0.24282837,  -0.21691895],
        [  4.5398865 ,   6.002594  ,   6.755127  , ...,   0.7687378 ,
           0.3908081 ,  -0.0743103 ],
        ...,
        [ 11.068024  ,  11.370697  ,  11.601593  , ...,   6.4604797 ,
           6.10553   ,   5.6962585 ],
        [ 10.924561  ,  11.093201  ,  11.106964  , ...,   6.7608337 ,
           6.3688354 ,   5.893799  ],
        [ 10.4635315 ,  10.656097  ,  10.687378  , ...,   6.98526   ,
           6.438385  ,   5.7952576 ]],

       [[ -3.9871826 ,  -2.2312012 ,  -0.10736084, ...,   0.89382935,
           0.7470703 ,   0.5819397 ],
        [ -3.267456  ,  -1.7615662 ,   0.26480103, ...,   1.2296143 ,
           1.0869751 ,   0.9006653 ],
        [ -2.510498  ,  -1.0487671 ,   0.8434143 , ...,   1.7808533 ,
           1.5757751 ,   1.333313  ],
...
        [ -5.597534  ,  -5.6144714 ,  -5.571747  , ..., -12.355209  ,
         -12.042709  , -10.764862  ],
        [ -4.3812256 ,  -4.185364  ,  -4.164551  , ..., -11.671097  ,
         -11.669113  , -10.961929  ],
        [ -3.828186  ,  -3.6273499 ,  -3.5612183 , ..., -11.34166   ,
         -11.203842  , -10.546616  ]],

       [[ -3.83609   ,  -4.4654846 ,  -4.9683228 , ..., -10.876282  ,
         -10.68692   , -10.607666  ],
        [ -3.6767273 ,  -4.2080383 ,  -4.685608  , ..., -10.602173  ,
         -10.3715515 , -10.350037  ],
        [ -3.417633  ,  -4.0882263 ,  -4.5871887 , ...,  -9.734863  ,
          -9.421234  ,  -9.683075  ],
        ...,
        [ -5.0591125 ,  -5.2108154 ,  -5.3341064 , ..., -10.172272  ,
         -10.014069  ,  -9.884644  ],
        [ -4.262726  ,  -4.297333  ,  -4.4933167 , ..., -10.021362  ,
          -9.660004  ,  -9.021179  ],
        [ -4.3698425 ,  -4.0772095 ,  -4.0071716 , ...,  -9.189972  ,
          -8.782623  ,  -8.094147  ]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 1979-11-01 ... 2019-04-01T18:00:00
  * lat        (lat) float32 40.35 41.05 41.75 42.46 ... 47.37 48.07 48.77 49.47
  * lon        (lon) float32 270.0 270.7 271.4 272.1 ... 287.6 288.3 289.0 289.7
    dayofyear  (time) int64 305 305 305 305 306 306 306 ... 90 90 90 91 91 91 91

# Import and Reshape Vals Now

In [8]:
infile = open("../New_EllipseVals/ephi10_79.p", 'rb')
ephi10 = pickle.load(infile)
infile.close()

infile = open("../New_EllipseVals/ephi_ratio10_79.p", 'rb')
rat10 = pickle.load(infile)
infile.close()

infile = open("../New_EllipseVals/ephi_size10_79.p", 'rb')
size10 = pickle.load(infile)
infile.close()

infile = open("../New_EllipseVals/ephi_cenlat10_79.p", 'rb')
cenlat10 = pickle.load(infile)
infile.close()

infile = open("../New_EllipseVals/ephi_cenlon10_79.p", 'rb')
cenlon10 = pickle.load(infile)
infile.close()

infile = open("../New_EllipseVals/ephi_wind10_79.p", 'rb')
wind10 = pickle.load(infile)
infile.close()

In [75]:
wind1 = np.nanmean(wind10, axis=0)
ephi1 = np.nanmean(ephi10, axis=0)
rat1 = np.nanmean(rat10, axis=0)
size1 = np.nanmean(size10, axis=0)
cenlt1 = np.nanmean(cenlat10, axis=0)
cenln1 = np.nanmean(cenlon10, axis=0)

In [9]:
wind = np.reshape(wind10, (24320))
ephi = np.reshape(ephi10, (24320))
rat = np.reshape(rat10, (24320))
size = np.reshape(size10, (24320))
cenlt= np.reshape(cenlat10, (24320))
cenln= np.reshape(cenlon10, (24320))

In [ ]:
print(scipy.stats.kendalltau(wind, ephi, nan_policy='omit'))
print(scipy.stats.kendalltau(wind, rat, nan_policy='omit'))
print(scipy.stats.kendalltau(wind, size, nan_policy='omit'))
print(scipy.stats.kendalltau(wind, cenlt, nan_policy='omit'))
print(scipy.stats.kendalltau(wind, cenln, nan_policy='omit'))

In [ ]:
##this is a manual calculation of a pearson corr coeff (https://www.socscistatistics.com/tests/pearson/)
def correlation_coefficient(T1, T2):
    numerator = np.nanmean((T1 - np.nanmean(T1)) * (T2 - np.nanmean(T2)))
    denominator = np.nanstd(T1) * np.nanstd(T2)
    if denominator == 0:
        return 0
    else:
        result = numerator / denominator
        return result

In [ ]:
print(correlation_coefficient(wind, ephi))
print(correlation_coefficient(wind, rat))
print(correlation_coefficient(wind, size))
print(correlation_coefficient(wind, cenlt))
print(correlation_coefficient(wind, cenln))

In [ ]:
print(correlation_coefficient(cenln,rat))
print(correlation_coefficient(cenln,ephi))
print(correlation_coefficient(cenln,size))
print(correlation_coefficient(cenln,cenlt))

In [10]:
t1 = np.nanmean(cap_anom_t.values,axis=1)
ta = np.nanmean(t1 ,axis=1)

t2 = np.nanmean(gl_anom_t.values,axis=1)
tg = np.nanmean(t2 ,axis=1)

g1 = np.nanmean(cap_anom_gph.values,axis=1)
ga = np.nanmean(g1 ,axis=1)
##calculation of daily mean cap anomalies

In [63]:
t3 = np.nanmean(t_gl.values,axis=1)
tgl = np.nanmean(t3 ,axis=1)

In [80]:
t4 = tgl.reshape((40,608))
g4 = ga.reshape((40,608))

In [81]:
tgl1 = np.nanmean(t4, axis =0)
ga1 = np.nanmean(g4, axis =0)

In [ ]:
print(correlation_coefficient(ga, wind))
print(correlation_coefficient(ga, ephi))
print(correlation_coefficient(ga, rat))
print(correlation_coefficient(ga, size))
print(correlation_coefficient(ga, cenlt))
print(correlation_coefficient(ga, cenln))

In [11]:
tempre = np.empty((40,552))
tempre[:] = np.nan

te = ta.reshape((40,608))

for i in range(0,40,1):
    tempre[i,:] = te[i,0:552]

tempre.reshape((22080,))

array([-0.55129671, -0.38757467, -0.19995734, ..., -0.90153688,
       -0.65570319, -0.50292814])

In [12]:
tempre2 = np.empty((40,552))
tempre2[:] = np.nan

te = tg.reshape((40,608))

for i in range(0,40,1):
    tempre2[i,:] = te[i,0:552]

tempre2.reshape((22080,))

array([ 7.44997978,  7.75497961,  5.99541235, ..., -6.60532284,
       -6.94425106, -6.22089291])

In [13]:
w = np.empty((40,552))
w[:] = np.nan

e = np.empty((40,552))
e[:] = np.nan

s = np.empty((40,552))
s[:] = np.nan

r = np.empty((40,552))
r[:] = np.nan

ct = np.empty((40,552))
ct[:] = np.nan

cn = np.empty((40,552))
cn[:] = np.nan

g = np.empty((40,552))
g[:] = np.nan

ge = ga.reshape((40,608))

for i in range(0,40,1):
    w[i,:] = wind10[i,56:]
    e[i,:] = ephi10[i,56:]
    s[i,:] = size10[i,56:]
    r[i,:] = rat10[i,56:]
    ct[i,:] = cenlat10[i,56:]
    cn[i,:] = cenlon10[i,56:]
    g[i,:] = ge[i,56:]

w.reshape((22080,))
e.reshape((22080,))
s.reshape((22080,))
r.reshape((22080,))
ct.reshape((22080,))
cn.reshape((22080,))
g.reshape((22080,))

array([ 116.91026306,  109.42657471,  108.72285461, ..., -475.53741455,
       -468.15454102, -470.47845459])

In [ ]:
w = w[~np.isnan(w)]
e = e[~np.isnan(e)]
s = s[~np.isnan(s)]
r = r[~np.isnan(r)]
ct = ct[~np.isnan(ct)]
cn = cn[~np.isnan(cn)]
g = g[~np.isnan(g)]

In [ ]:
print(len(g))

In [ ]:
print(correlation_coefficient(tempre, w))
print(correlation_coefficient(tempre, e))
print(correlation_coefficient(tempre, r))
print(correlation_coefficient(tempre, s))
print(correlation_coefficient(tempre, ct))
print(correlation_coefficient(tempre, cn))

In [ ]:
##test multilinear regression

In [82]:
import pandas as pd

data = {'wind': np.ndarray.tolist(wind1),
        'ephi': np.ndarray.tolist(ephi1),
        'rat': np.ndarray.tolist(rat1),
        'size': np.ndarray.tolist(size1),
        'cenlat': np.ndarray.tolist(cenlt1),
        'cenlon': np.ndarray.tolist(cenln1),
        'temp': np.ndarray.tolist(tgl1),
        'gph': np.ndarray.tolist(ga1)
        }
#data 
df = pd.DataFrame(data)
df = df.dropna()
#df

In [83]:
df_norm = (df - df.mean())
df_norm

,wind,ephi,rat,size,cenlat,cenlon,temp,gph
0,-1.793181,16.166455,0.010931,-6.527054e+06,7.622356,5.824317,8.664267,30.536285
1,-1.682468,22.572913,0.008596,-6.192786e+06,7.490755,4.001203,8.634146,21.475296
2,-1.587943,19.641788,0.015691,-6.274436e+06,7.634300,3.886376,8.257803,17.261034
3,-1.266700,12.436713,0.020197,-6.089756e+06,7.666230,5.190552,8.191123,11.296602
4,-1.323162,9.828738,0.020385,-5.809388e+06,7.485693,6.515353,8.013876,10.397055
...,...,...,...,...,...,...,...,...
603,-15.462299,-14.533808,-0.098347,-1.891512e+07,-5.937147,20.012743,5.067954,4.293884
604,-15.316578,-8.674220,-0.096076,-1.899083e+07,-5.421489,19.964729,5.410605,-5.058428
605,-15.349710,-14.784542,-0.150460,-1.889054e+07,-8.273965,18.375660,5.099112,-6.912093
606,-14.884450,-9.975683,-0.108183,-1.912738e+07,-5.581311,14.014725,4.494132,4.932523


In [84]:
x = df_norm[['ephi','rat','cenlat','cenlon','size','wind','gph']]
y = df_norm['temp']

In [85]:
from statsmodels.stats.outliers_influence import variance_inflation_factor 

#VIF dataframe 
vif_data = pd.DataFrame() 
vif_data["feature"] = x.columns 
  
# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(x.values, i) 
                          for i in range(len(x.columns))] 
  
print(vif_data)


  feature        VIF
0    ephi   2.337356
1     rat   1.352545
2  cenlat   3.141235
3  cenlon   2.556808
4    size  19.204519
5    wind  25.348270
6     gph   1.012404


In [91]:
x = df_norm[['rat','cenlat','wind','gph']]
y = df_norm['temp']

In [92]:
import statsmodels.api as sm

# with statsmodels
x = sm.add_constant(x) # adding a constant
 
model = sm.GLM(y, x).fit()
predictions = model.predict(x) 
 
print_model = model.summary()
print(print_model)

                 Generalized Linear Model Regression Results                  
Dep. Variable:                   temp   No. Observations:                  608
Model:                            GLM   Df Residuals:                      603
Model Family:                Gaussian   Df Model:                            4
Link Function:               identity   Scale:                          2.4097
Method:                          IRLS   Log-Likelihood:                -1127.6
Date:                Thu, 26 Oct 2023   Deviance:                       1453.0
Time:                        16:00:55   Pearson chi2:                 1.45e+03
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       6.674e-14      0.063   1.06e-12      1.0